## Defining functions to calculate inner and tensor products

In [11]:
import numpy as np

In [12]:
def colonProduct(lval,rval):
    
    """
    Function for calculating colon product of two second rank tensors

    Parameters
    ----------

    lval: np.ndarray
        second rank tensor in an array with a shape of (6,)

    rval: np.ndarray
        second rank tensor in an array with a shape of (6,)

    Returns
    -------

    int, float : the Colon product

    """
    
    return (lval[0]*rval[0] + lval[1]*rval[1] + lval[2]*rval[2]
            + 2*(lval[3]*rval[3] + lval[4]*rval[4] + lval[5]*rval[5]))



In [13]:
def colonProduct_T4_T2(lval,rval):

    """
    Function for calculating Colon product of a fourth rank tensor and a second rank tensor

    Parameters
    ----------

    lval: np.ndarray
        fourth rank tensor in an array with a shape of (6,6)

    rval: np.ndarray
        second rank tensor in an array with a shape of (6,)

    Returns
    -------

    np.ndarray : the Colon product in an array with a shape of (6,)

    """
    
    return lval @ rval



In [14]:
def dyadicProduct(lval,rval):
    
    """
    Function for calculating dyadic product of two second rank tensors

    Parameters
    ----------

    lval: np.ndarray
        second rank tensor in an array with a shape of (6,)

    rval: np.ndarray
        second rank tensor in an array with a shape of (6,)

    Returns
    -------

    np.ndarray : the dyadic product in an array with a shape of (6,6)

    """    
    
    res = np.zeros([6,6])
    for i in np.arange(6):
        for j in np.arange(6):
            if j>2:
                res[i,j] = 2 * lval[i] * rval[j]
            else:
                res[i,j] = lval[i] * rval[j]
    return res

In [15]:
I = np.array([1,1,1,0,0,0])

def invariant(sigm):

    """
    Function for calculating stress tensor invariant J1, deviatoric stress tensor invariant J2D and Lode angle

    Parameters
    ----------

    sigm: np.ndarray
        a second rank stress tensor in an array with a shape of (6,)


    Returns
    -------

    J1 : int, float
        the stress tensor invariant J1
        
    J2D : int, float
        the deviatoric stress tensor invariant J2D

    theta : float
        the Lode angle

    """
    
    
    sigma = np.array([[sigm[0],sigm[3],sigm[4]],
                 [sigm[3],sigm[1],sigm[5]],
                 [sigm[4],sigm[5],sigm[2]]])

    I1 = sigm[0] + sigm[1] + sigm[2]
    I2 = np.linalg.det(sigma[0:2,0:2]) + np.linalg.det(sigma[1:3,1:3]) + sigm[0]*sigm[2] - sigm[4]**2
    I3 = np.linalg.det(sigma)

    J1 = I1
    J2 = 1/2 * (I1**2 - 2*I2)
    J3 = 1/3 * (I1**3 - 3*I1*I2 + 3*I3)

    J2D = J2 - J1**2/6
    J3D = J3 - 2/3*J1*J2 + 2/27*J1**3

    if J2D == 0 and I1 ==0:
        print('No Lode Angle!')
        theta = 0
    
    else:
        theta = 1/3 * np.real(np.emath.arcsin(-3 * np.sqrt(3) / 2 *J3D / np.emath.power(J2D,1.5)))
        return J1, J2D, theta

In [16]:
def C_el(Model):
    
    """
    Function for obtaining an elastic stiffness tensor with given model parameters K and G

    Parameters
    ----------

    Model: a Model object
        A Model object with K and G values assigned.


    Returns
    -------

    np.ndarray : an elastic stiffness tensor in an array with a shape of (6,6)

    """
    temp1 = Model.K + 4/3*Model.G
    temp2 = Model.K - 2/3*Model.G
    temp3 = 2*Model.G
    
    return np.array([[temp1,temp2,temp2,0,0,0],
                   [temp2,temp1,temp2,0,0,0],
                   [temp2,temp2,temp1,0,0,0],
                   [0,0,0,temp3,0,0],
                   [0,0,0,0,temp3,0],
                   [0,0,0,0,0,temp3]])

In [17]:
def constraint(incr,Load_tag,inc_frac):

    """
    Function for constructing Bardet's constraints matrices

    Parameters
    ----------

    incr: float
        deps_a

    Load_tag: {100, 110}
        If 100, undrained shearing
        If 110, drained shearing
    
    inc_frac: float
        Must be within the interval [0,1], the portion of new loading step to be applied
    

    Returns
    -------

    E_Bardet : np.ndarray
        Constraint matrix for strain in an array with a shape of (6,6)
        
    S_Bardet : np.ndarray
        Constraint matrix for stress in an array with a shape of (6,6)

    dY_Bardet : np.ndarray
        loading vector in an array with a shape of (6,)

    """
    
    
    dX = incr * inc_frac

    if Load_tag == 110: #drained triaxial
        S = np.array([[1,0,0,0,0,0],
                    [0,1,0,0,0,0],
                    [0,0,0,1,0,0],
                    [0,0,0,0,1,0],
                    [0,0,0,0,0,1],
                    [0,0,0,0,0,0]])

        E =  np.array([[0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,1,0,0,0]])

    if Load_tag == 100: #undrained triaxial
        S = np.array([[1,-1,0,0,0,0],
                    [0,0,0,1,0,0],
                    [0,0,0,0,1,0],
                    [0,0,0,0,0,1],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0]])

        E =  np.array([[0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [0,0,0,0,0,0],
                    [1,1,1,0,0,0],
                    [0,0,1,0,0,0]])

    dY = np.array([0,0,0,0,0,dX])
    E_Bardet = E
    S_Bardet = S
    dY_Bardet = dY

    return E_Bardet, S_Bardet, dY_Bardet

In [18]:
def Bardet(Cel, incr, Load_tag, inc_frac):
   
    """
    Function for solving Bardet's equation to obtain $\dot{\epsilon}$ and $\dot{\sigma}$ 

    Parameters
    ----------
    
    Cel: np.ndarray
        a stiffness tensor in an array with a shape of (6,6)
    
    incr: float
        deps_a

    Load_tag: {100, 110}
        If 100, undrained shearing
        If 110, drained shearing
    
    inc_frac: float
        Must be within the interval [0,1], the portion of new loading step to be applied
    

    Returns
    -------

    deps_trial : np.ndarray
        strain increment in an array with a shape of (6,)        
        
    dsigm_trial : np.ndarray
        stress increment an array with a shape of (6,)

    """
        
    E_Bardet, S_Bardet, dY_Bardet = constraint(incr, Load_tag, inc_frac)
    
    A_Bardet = S_Bardet @ Cel + E_Bardet
    deps_trial = np.linalg.solve(A_Bardet,dY_Bardet)
    dsigm_trial = Cel @ deps_trial
    
    return deps_trial, dsigm_trial

In [19]:
def runningModule(State, Model, Load_tag, Load_limit, Load_cyclic, Load_num_cyc, Load_incr=1e-5, ax = None):
    
    """
    Function for solving Bardet's equation to obtain $\dot{\epsilon}$ and $\dot{\sigma}$ 

    Parameters
    ----------
    
    State: a State object
    
    Model: a Model object
    
    Load_tag: {100, 110}
        If 100, undrained shearing
        If 110, drained shearing
    
    Load_limit: float
        eps_a limit
    
    Load_cyclic: {0, 1}
        If 0, monotonic
        If 1, cyclic
    
    Load_num_cyc: int
        Number of cycles
    
    Load_incr: float, default 1e-5
        deps_a 
    
    ax: matplotlib.axes, optional
        existing sub-plots to be used 


    Returns
    -------

    ax: matplotlib.axes


    """    
    
    
    def loading(State):
        return  2/3 * (State.strain[2]-State.strain[0])

    # Initialize output space
    class Output:
        stress = State.stress.copy()
        strain = State.strain.copy()
        others = State.strainp.copy()
    
    Load_value = loading(State)
    
    NP = 10                      # Input intervals

    for load_cycle in np.arange(Load_num_cyc):
        Load_step = 0
        while Load_value * np.sign(Load_incr) < Load_limit * np.sign(Load_incr):
            State = constitutiveDriver(Model, Load_incr, Load_tag, State)
            if Load_step % NP ==0:
                Output.stress = np.vstack((Output.stress, State.stress))
                Output.strain = np.vstack((Output.strain, State.strain))
                Output.others = np.vstack((Output.others, State.strainp))
            Load_step = Load_step + 1
            Load_value = loading(State)
        
        
        Load_incr = -Load_incr
        Load_limit = -Load_limit
        
        while Load_value * np.sign(-Load_incr) > Load_limit * np.sign(-Load_incr) and Load_cyclic == 1:
            State = constitutiveDriver(Model, Load_incr, Load_tag, State)
            if Load_step % NP ==0:
                Output.stress = np.vstack((Output.stress, State.stress))
                Output.strain = np.vstack((Output.strain, State.strain))
                Output.others = np.vstack((Output.others, State.strainp))
            Load_step = Load_step + 1
            Load_value = loading(State)
        
        
        Load_incr = -Load_incr
        Load_limit = -Load_limit
        
        while Load_value * np.sign(Load_incr) < 0 and Load_cyclic == 1:
            State = constitutiveDriver(Model, Load_incr, Load_tag, State)
            if Load_step % NP ==0:
                Output.stress = np.vstack((Output.stress, State.stress))
                Output.strain = np.vstack((Output.strain, State.strain))
                Output.others = np.vstack((Output.others, State.strainp))
            Load_step = Load_step + 1
            Load_value = loading(State)
        
    
    
    # Plotting
    
    p = (Output.stress[:,0] + Output.stress[:,1] + Output.stress[:,2])/3
    q = Output.stress[:,2] - Output.stress[:,0]
    eps_v = (Output.strain[:,0] + Output.strain[:,1] + Output.strain[:,2]) * 100
    eps_q = 2/3 * (Output.strain[:,2] - Output.strain[:,0]) * 100
    p_total = p[0] + q/3
    u = p_total - p
    
    if ax is None: 
        fig, ax = plt.subplots(2, 2, figsize=(12, 8))
    
    ax[0,0].plot(eps_q,q)
    ax[0,1].plot(p,q)
    if Load_tag == 110:
        ax[1,0].plot(eps_q,eps_v)
        ax[1,1].plot(p,eps_v)
    if Load_tag == 100:
        ax[1,0].plot(eps_q,u)
        ax[1,1].plot(p,u)


    ax[0,0].set_xlabel(f'$\epsilon_q$ (%)')
    ax[0,0].set_ylabel("q (kPa)")

    ax[0,1].set_xlabel("p (kPa)")
    ax[0,1].set_ylabel("q (kPa)")
    ax[0,1].set_xlim(0)
    ax[0,1].set_aspect(1, adjustable = "datalim")
    ax[0,1].apply_aspect()
    xmin01, xmax01, ymin01, ymax01 = ax[0,1].axis() 

    
    ax[1,0].set_xlabel(f'$\epsilon_q$ (%)')
    if Load_tag == 110:
        ax[1,0].set_ylabel(f'$\epsilon_v$ (%)')
    if Load_tag == 100:
        ax[1,0].set_ylabel(f'u (kPa)')
    
    
    ax[1,1].set_xlabel("p (kPa)")
    if Load_tag == 110:
        ax[1,1].set_ylabel(f'$\epsilon_v$ (%)')
    if Load_tag == 100:
        ax[1,1].set_ylabel(f'u (kPa)')    
    ax[1,1].set_xlim(xmin01, xmax01)
    
    ax[0,0].grid(alpha=0.5)
    ax[0,1].grid(alpha=0.5)
    ax[1,0].grid(alpha=0.5)
    ax[1,1].grid(alpha=0.5)
    
    return ax
    